# Adds cloud mask feature to downloaded eopatches

This notebook applies the same post-processing as `s2cloudless` on the `CLP` mask downloaded from Sentienl-Hub. To avoid doing this, the `CLM` from Sentinel-Hub can be used directly (should lead to negligible differences).

In [1]:
import os
import time

from eolearn.core import EOPatch, SaveTask, FeatureType, OverwritePermission, \
    EOTask, LoadTask, LinearWorkflow, EOExecutor

import numpy as np

from s2cloudless import S2PixelCloudDetector

import geopandas as gpd

In [2]:
class CloudMasking(EOTask):
    def __init__(self, clm_feature=(FeatureType.MASK, 'CLM')):
        self.clm_feature = next(self._parse_features(clm_feature)())
        self.s2_cd = S2PixelCloudDetector(average_over=24)
            
    def execute(self, eopatch):
        clc = self.s2_cd.get_mask_from_prob(eopatch.data['CLP'].squeeze()/255.)
        eopatch[self.clm_feature] = clc[..., np.newaxis]
        return eopatch

In [3]:
load_task = LoadTask('s3://bucket-name/data/Lithuania/eopatches/2019/', lazy_loading=True)

clm_feature = (FeatureType.MASK, 'CLM')
clm_task = CloudMasking(clm_feature)

save_task = SaveTask('s3://bucket-name/data/Lithuania/eopatches/2019/', 
                     features=[clm_feature], overwrite_permission=OverwritePermission.ADD_ONLY)

In [4]:
workflow = LinearWorkflow((load_task, 'load'), (clm_task, 'cloud_masking'), (save_task, 'save'))

In [5]:
eopatches = gpd.read_file('./eopatches.geojson')['eopatch_name'].values

In [6]:
execution_args = [{load_task: { 'eopatch_folder': eopatch },
                   save_task: { 'eopatch_folder': eopatch }
                  } for eopatch in eopatches]

In [7]:
len(execution_args)

938

In [8]:
execution_args[0]

{<eolearn.core.core_tasks.LoadTask at 0x7f18a84a07b8>: {'eopatch_folder': 'eopatch-0000'},
 <eolearn.core.core_tasks.SaveTask at 0x7f18a84a0898>: {'eopatch_folder': 'eopatch-0000'}}

In [9]:
executor = EOExecutor(workflow, execution_args, save_logs=True, logs_folder='.')

In [10]:
executor.run(workers=36)

In [13]:
%matplotlib

executor.make_report()

print('Report was saved to location: {}'.format(executor.get_report_filename()))

Using matplotlib backend: agg
Report was saved to location: ./eoexecution-report-2020_06_10-12_23_47/report.html
